In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Add the project's files to the python path
# file_path = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))  # for .py script
file_path = os.path.dirname(os.path.abspath(''))  # for .ipynb notebook
sys.path.append(file_path)

In [ ]:
import torch
import laspy
import numpy as np

from src.data import Data, InstanceData
from torch_geometric.nn.pool.consecutive import consecutive_cluster
from src.transforms import *

FOR_Instance_num_classes = 3
ID2TRAINID = np.asarray([
    FOR_Instance_num_classes,   # 0 Unclassified        ->  3 Ignored
    1,                          # 1 Low vegetation      ->  1 Low vegetation
    0,                          # 2 Terrain             ->  0 Ground
    FOR_Instance_num_classes,   # 3 Out-points          ->  3 Ignored
    2,                          # 4 Stem                ->  2 Tree
    2,                          # 5 Live branches       ->  2 Tree
    2,                          # 6 Woody branches      ->  2 Tree
])

FOR_Instance_CLASS_NAMES = [
    'Ground',
    'Low vegetation',
    'Tree',
    'Ignored']

FOR_Instance_CLASS_COLORS = np.asarray([
    [243, 214, 171],    # Ground
    [204, 213, 174],    # Low vegetation
    [ 70, 115,  66],    # Tree
    [  0,   0,   0]     # Ignored
])

filepaths = [
    "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/CULS/plot_3_annotated.las",
    "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/NIBIO/plot_10_annotated.las",
    "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/RMIT/train.las",
    "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/SCION/plot_35_annotated.las",
    "/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/TUWIEN/train.las"
]

In [ ]:
def read_FORinstance_plot(filepath, xyz=True, intensity=True, semantic=True, instance=True, remap=True, max_intensity=None):
    """
    Read a FORinstance plot from a LAS file and return the data object.
    """
    data = Data()
    las = laspy.read(filepath)

    if xyz:
        pos = torch.stack([
            torch.as_tensor(np.array(las[axis]))
            for axis in ["X", "Y", "Z"]], dim=-1)
        pos *= las.header.scale
        pos_offset = pos[0]
        data.pos = (pos - pos_offset).float()
        data.pos_offset = pos_offset

    intensity_remaped = True
    if intensity:
        data.intensity = torch.FloatTensor(
            las['intensity'].astype('float32')
        )
        if intensity_remaped:
            if max_intensity is None:
                max_intensity = data.intensity.max()
            data.intensity = data.intensity.clip(min=0, max=max_intensity) / max_intensity

    if semantic:
        y = torch.LongTensor(np.array(las['classification']))
        data.y = torch.from_numpy(ID2TRAINID)[y] if remap else y

    if instance:
        idx = torch.arange(data.num_points)
        obj = torch.LongTensor(np.array(las['treeID']))
        
        y = torch.LongTensor(np.array(las['classification']))
        y = torch.from_numpy(ID2TRAINID)[y] if remap else y

        if remap:
            ground_mask = (obj == 0) & (y == 0)
            low_veg_mask = (obj == 0) & (y == 1)
            if low_veg_mask.any() or ground_mask.any():
                ground_instance_label = obj.max().item() + 1
                low_veg_instance_label = ground_instance_label + 1
                obj[ground_mask] = ground_instance_label
                obj[low_veg_mask] = low_veg_instance_label

        obj = consecutive_cluster(obj)[0]
        count = torch.ones_like(obj)

        data.obj = InstanceData(idx, obj, count, y, dense=True)
    
    return data

In [ ]:
def apply_transform(data, transform, *args, **kwargs):
    """
    Helper function to apply a transform to the data and return the transformed data.
    """
    return transform(*args, **kwargs)(data)

def pre_transform(data, param_set):
    """
    Pre-transform the data with the specified parameters and return the transformed data element as a nag.

    The function applies a series of transformations to the input dataset and returns the transformed data.

    Parameters:
    data (object): A data object to be transformed and evaluated.
    param_set (tuple): A tuple containing the parameters for the transformations in the following order:
                          (voxel_size, k, r_max, threshold, scale, features, k_adj_graph, w, features_to_x, regularization,
                            spatial_weight, cutoff, iterations, k_adjacency)

    Returns:
    object: nag, the transformed data element.
    """

    # Extract parameters
    (voxel_size, k, r_max, threshold, scale, features, k_adj_graph, w, features_to_x, 
         regularization, spatial_weight, cutoff, iterations, k_adjacency) = param_set

    data = data.clone()
    data = apply_transform(data, GridSampling3D, size=voxel_size, hist_key='y', hist_size=FOR_Instance_num_classes + 1)
    data = apply_transform(data, KNN, k, r_max)
    data = apply_transform(data, GroundElevation, threshold, scale)
    data = apply_transform(data, PointFeatures, keys=features_to_x)
    data = apply_transform(data, AdjacencyGraph, k_adj_graph, w)
    data = apply_transform(data, AddKeysTo, keys=features_to_x, to='x', delete_after=False)
    nag = apply_transform(data, CutPursuitPartition, regularization=regularization, \
                                   spatial_weight=spatial_weight, cutoff=cutoff, iterations=iterations, k_adjacency=k_adjacency)

    return nag

In [ ]:
def apply_params_and_vis(data, param_set):
    """
    Apply the parameters to the data and visualize the result.
    """
    nag = pre_transform(data, param_set)
    print('Tree iou: ', nag[1].semantic_segmentation_oracle(FOR_Instance_num_classes)['iou_per_class'][2].item())
    print(nag[1].instance_segmentation_oracle(FOR_Instance_num_classes))
    print(nag[1].panoptic_segmentation_oracle(FOR_Instance_num_classes))

    nag.show(class_names=FOR_Instance_CLASS_NAMES, class_colors=FOR_Instance_CLASS_COLORS)

In [ ]:
"""
0: CULS
1: NIBIO
2: RMIT
3: SCION
4: TUWIEN
"""
data_CULS = read_FORinstance_plot(filepaths[0], instance=True)
data_NIBIO = read_FORinstance_plot(filepaths[1], instance=True)
data_RMIT = read_FORinstance_plot(filepaths[2], instance=True)
data_SCION = read_FORinstance_plot(filepaths[3], instance=True)
data_TUWIEN = read_FORinstance_plot(filepaths[4], instance=True)

data_NIBIO_1 = read_FORinstance_plot('/home/valerio/git/superpoint_transformer_vschelbi/data/FORinstance/raw/NIBIO/plot_1_annotated.las', instance=True)

In [ ]:
params = [0.1, 25, 3, 5, 20, ['intensity', 'linearity', 'planarity', 'scattering', 'verticality', 'elevation'], 5, 1, ['intensity', 'linearity', 'planarity', 'scattering', 'verticality', 'elevation'], [0.1, 0.2], [0.1, 0.01], [10, 30], 15, 5]

#(voxel_size, k, r_max, threshold, scale, features, k_adj_graph, w, features_to_x, 
#         regularization, spatial_weight, cutoff, iterations, k_adjacency) = params

In [ ]:
apply_params_and_vis(data_CULS, params)

In [ ]:
apply_params_and_vis(data_NIBIO, params)

In [ ]:
apply_params_and_vis(data_NIBIO_1, params)

In [ ]:
apply_params_and_vis(data_RMIT, params)

In [ ]:
apply_params_and_vis(data_SCION, params)

In [ ]:
apply_params_and_vis(data_TUWIEN, params)